<a href="https://colab.research.google.com/github/Juhyunn0/Autonomous-control-using-automated-image-analysis/blob/main/Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')

In [ ]:
! pip install pyserial

In [ ]:
import time
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import os
import cv2
import numpy as np
import math
import serial
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
from core import connect
from IPython import display
import pandas as pd

JJ_path = '/content/drive/MyDrive/Amour_R&D_Juhyun_Jung(JJ)'
path = JJ_path

os.chdir('/content/drive/MyDrive/Amour_R&D_Juhyun_Jung(JJ)')

# 1. Check the HSV values

In [ ]:
def get_HSV_in_image(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONUP:
        hsv_image = cv2.cvtColor(param['image'], cv2.COLOR_BGR2HSV)
        hsv_value = hsv_image[y, x, :]
        print("HSV values at clicked point: Hue={}, Saturation={}, Value={}".format(hsv_value[0], hsv_value[1], hsv_value[2]))

    if event == cv2.EVENT_RBUTTONUP:
        threshold = 20
        bgr_value = param['image'][y, x, :]
        lower = np.array([max(0, bgr_value[0] - threshold),
                          max(0, bgr_value[1] - threshold),
                          max(0, bgr_value[2] - threshold)])
        upper = np.array([min(255, bgr_value[0] + threshold),
                          min(255, bgr_value[1] + threshold),
                          min(255, bgr_value[2] + threshold)])
        mask = cv2.inRange(param['image'], lower, upper)
        range_image = cv2.bitwise_and(param['image'], param['image'], mask=mask)
        cv2.imshow("range_image", range_image)
        cv2.waitKey(0)

origin_image = cv2.imread("C:\\Users\\Nikonb\\Python code\\data\\testimage5.jpg")

param = {
    'image': origin_image
}
cv2.imshow('image', origin_image)
cv2.setMouseCallback('image', get_HSV_in_image, param)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 2. Image processing

In [1]:
def process_image(image_path, counter, formatted_time_info, curr_image):
    max_attempts = 10
    attempt = 0
    wait_time = 0.5

    next_image = None
    while attempt < max_attempts:
        next_image = cv2.imread(image_path)
        if next_image is not None:
            break
        attempt += 1
        time.sleep(wait_time)

    if next_image is None:
        print(f"Error: Unable to read the image file {image_path} after {max_attempts} attempts")
        return

    if curr_image is None:
        curr_image = next_image
        cell_count = 0
    else:
        diff = cv2.absdiff(curr_image, next_image)
        diff_rgb = cv2.cvtColor(diff, cv2.COLOR_BGR2RGB)
        diff_image_path = f"{save_path}0. Difference two Images/diff_image  {formatted_time_info} seconds.jpg"
        cv2.imwrite(diff_image_path, diff)
        hsv = cv2.cvtColor(diff, cv2.COLOR_BGR2HSV)
        lower_color = np.array([0, 0, 24])
        upper_color = np.array([0,0,150])

        mask = cv2.inRange(hsv, lower_color, upper_color)
        result = cv2.bitwise_and(diff_rgb, diff_rgb, mask=mask)
        gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
        cnts = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if len(cnts) == 2 else cnts[1]

        min_contour_area = 0.1
        max_contour_area = 10000  # maximum area
        cell_count = 0
        font_scale = 0.5

        for i, c in enumerate(cnts):
            area = cv2.contourArea(c)
            if min_contour_area <= area <= max_contour_area:
                cell_count += 1
                cv2.drawContours(result, [c], -1, (0, 255, 0), 2)
                M = cv2.moments(c)
                if M["m10"] != 0 and M["m00"] != 0:
                    cX = int(M["m10"] / M["m00"])
                    cY = int(M["m01"] / M["m00"])
                    # Display area of the contour
                    cv2.putText(result, f"Area: {area}", (cX - 50, cY - 20),
                                cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), 2)
                    # Display count centered at the centroid of the contour
                    cv2.putText(result, f"Count: {cell_count}", (cX - 30, cY + 30),
                                cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), 2)

        print(f"Number of cells: {cell_count}")

        alpha = 1
        beta = 1
        combined_image = cv2.addWeighted(diff, alpha, result, beta, 0)
        combined_image_path = f"{save_path}3. Combined Images/combined_image  {formatted_time_info} seconds.jpg"
        cv2.imwrite(combined_image_path, combined_image)


    original_image_path = f"{save_path}1. Original Images/original_image  {formatted_time_info} seconds.jpg"
    cv2.imwrite(original_image_path, next_image)
    return cell_count,next_image

# 3. Save Data

In [ ]:
def save_data(time_lst, data,name,file_name):
    start = len(time_lst)-len(data)
    df = pd.DataFrame({
      'Time':time_lst[start:],
      name:data
    })
    df.to_csv(save_path+'csv files/'+f'{month}{day}_'+name+f'{file_name}.csv', index=False)

# 4 Final Code

In [ ]:
Lab_path = "C:/Users/Nikonb/Python code/"
JJ_path = "C:/Users/juhye/Nikonb/Python code/"
save_path = Lab_path

# Get today's date
today = datetime.now()

# Extract the month and day
month = today.strftime("%m")  # Full month name
day = today.day               # Day of the month

class ImageHandler(FileSystemEventHandler):
    def __init__(self):
        super().__init__()
        self.cell_count = 0
        self.Kp = 0.04
        self.Ki = 0.03
        self.Kd = 0.02
        # Initialize variables
        self.prev_error = 0
        self.integral = 0

        # Initialize pump connection
        self.portinfo = connect.getOpenPorts()
        self.baudrate = 9600
        self.conn = connect.Connection(port=str(self.portinfo[-1]), baudrate=self.baudrate, x=0, mode=0)

        # Open Connection to pump
        self.conn.openConnection()

        # Setup parameters for basic run
        self.units = 'μL/min'  # Flow rate units
        self.diameter = 4.66   # Syringe diameter in mm
        self.volume = 330     # Volume to dispense in μL
        self.delay = 0.0       # Delay in seconds

        # Communicate parameters to pump
        self.conn.setUnits(self.units)
        self.conn.setDiameter(self.diameter)
        self.conn.setVolume(self.volume)
        self.conn.setDelay(self.delay)

        # Start pump
        self.conn.startPump()

        self.desired_bead_count = 3  # Example desired number of beads
        self.flowrate= 1.5
        self.cell_count = 0
        self.flowrate_lst = [self.flowrate]
        self.beads_lst = []
        self.time_list = []
        self.start_time = time.time()
        current_time = time.time() - self.start_time
        self.time_list.append(current_time)
        self.conn.setRate(self.flowrate) # initial flow rate
        self.curr_image = None
        self.counter = 0
        self.upcounter =0


    def on_created(self, event):
        if event.is_directory:
            return None
        elif event.src_path.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):

            print(f'self.counter : {self.counter}')
            print(f'len(self.time_lst) : {len(self.time_list)}')
            formatted_time = time.strftime("%Y-%m-%d %H;%M;%S", time.localtime(time.time()))
            print(f"{self.counter}th image detected  ", formatted_time)
            self.counter = self.counter+1
              # time list
            current_time = time.time() - self.start_time
            self.time_list.append(current_time)
              # count the number of beads
            self.cell_count, self.curr_image = process_image(event.src_path, self.counter, current_time, self.curr_image)
            self.beads_lst.append(self.cell_count)


            # PID control
            print(f'current flow rate : {self.flowrate}')
            self.error = self.desired_bead_count - self.cell_count

            if self.counter%5==0: #reset integral and derivative
                self.integral=0
                self.derivative=0
            if self.error>7:
                self.error = 3
            self.integral += self.error
            self.derivative = self.error - self.prev_error
            print(f"error: {self.error}, "
                  f"self.error: {self.error} | self.integral: {self.integral} | self.derivative : {self.derivative}, \n"
                  f"So, {self.flowrate} + {self.Kp * self.error + self.Ki * self.integral + self.Kd * self.derivative} = "
                  f"{self.Kp * self.error + self.Ki * self.integral + self.Kd * self.derivative + self.flowrate}")
            self.flowrate = self.flowrate + self.Kp * self.error + self.Ki * self.integral + self.Kd * self.derivative
            print(f"before clip : {self.flowrate}", end=' | ')
            # saturation
            lower_boundary = 0.3
            upper_boundary = 5
            self.flowrate = np.clip(self.flowrate, lower_boundary,upper_boundary)
            print(f"after clip : {self.flowrate}")
            self.prev_error = self.error

#             self.flowrate = 1.5
            self.flowrate_lst.append(self.flowrate)

          # apply for syringe pump
            self.conn.setRate(self.flowrate)

            # plot the graph
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
            ax1.plot(self.time_list[1:],self.beads_lst[:],'r',label='#beads')
            ax1.scatter(self.time_list[1:],self.beads_lst[:],color='r')
            ax1.set_xlabel('time')
            ax1.set_ylabel('the number of beads')
            ax1.set_ylim(-0.2,15)
            ax1.axhline(self.desired_bead_count,color='black',linestyle='dashed',label='Objective')
            ax1.legend()

            ax2.plot(self.time_list[:],self.flowrate_lst[:],'r',label='output')
            ax2.scatter(self.time_list[:],self.flowrate_lst[:],color='r')
#             ax2.set_ylim(0.2,2.2)
            # plt.axhline(0,color='black',linestyle='dashed')
            ax2.set_xlabel('time')
            ax2.set_ylabel('flow rate')
            ax2.legend()
            plt.show()
            print('-' * 50)
            if self.counter%10==0:
                display.clear_output(wait=True)

            # save file
            if len(self.time_list) == 120:
                self.conn.setRate(0.3)
                file_name = f'_{self.Kp},{self.Ki},{self.Kd},[{lower_boundary},{upper_boundary}],'+str(input('Enter wait time : '))+'second'
                save_data(self.time_list,self.beads_lst,'beads',file_name)
                save_data(self.time_list,self.flowrate_lst,'flow rate',file_name)
                print('Saved')
            os.remove(event.src_path)  # delete image

if __name__ == "__main__":
    path = f"{save_path}data"
    event_handler = ImageHandler()
    observer = Observer()
    observer.schedule(event_handler, path, recursive=False)
    observer.start()

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()

#